In [0]:
import dataiku
from langchain.chains.question_answering import load_qa_chain
from dataiku.langchain.dku_llm import DKUChatLLM

KB_ID = "zQ92IhQ9"

client = dataiku.api_client()
project = client.get_default_project()

In [0]:
# Listing available LLMs
llm_list = project.list_llms()

for llm in llm_list:
    print(f"- {llm.description} (id: {llm.id})")

In [0]:
# Fill with your LLM id
LLM_ID = "openai:OpenAI-FA:gpt-4o"

In [0]:
# Preparing the Knowledge Bank, Vector store and LLM
kb = dataiku.KnowledgeBank(id=KB_ID, project_key=project.project_key)
vector_store = kb.as_langchain_vectorstore()
gpt_lc = DKUChatLLM(llm_id=LLM_ID, temperature=0)

In [0]:
# Create the question answering chain
chain = load_qa_chain(gpt_lc, chain_type="stuff")
query = "What will inflation in Europe look like and why?"


In [0]:
# Performing semantic (vector) search
search_results = vector_store.similarity_search(query)

for search_result in search_results:
    print(f"- {search_result.page_content}\n")

In [0]:
# ⚡ Get the results ⚡
resp = chain({"input_documents":search_results, "question": query})
print(resp["output_text"])